# Hydrogel-Cell Interaction

In this case, we only have the deformed shape of the spherical hydrogel as shown in the next few cells.

In [ ]:
import numpy as np
import scipy as sp
import scipy.sparse as spm
from scipy.io import loadmat, savemat

import matplotlib.pyplot as plt
from matplotlib import cm, colors
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection
import sys, time, os.path
from itertools import permutations

module_path = '../module/'
sys.path.append(module_path)
from SHUtil import SphCoord_to_CartCoord, CartCoord_to_SphCoord

import pyshtools
from SHUtil import SHCilmToVector, SHVectorToCilm, lmk2K, K2lmk
from SHUtil import plotfv, TransMat, l_coeffs, m_coeffs, LM_list
from ShElastic import calSmode, calUmode
from SHBV import generate_submat, visualize_Cmat, print_SH_mode

from scipy.sparse.linalg import lsqr, spsolve
from scipy.interpolate import RectBivariateSpline
from scipy.optimize import minimize

from Case05_utilities import Uvec2Tvec, Tvec2Uvec, vismesh
from Case05_utilities import SHVec2mesh, SHmesh2Vec, visSHVec, visSH3d
from Case05_utilities import SHvec_rtoc, SHvec_ctor

## Input information

In [ ]:
############################# change inputs here #################################
# Data file name
datadir = '' #'../testdata'
smoothed = '' # 'smoothed_lmax20_lwork15'
dilated = '' #'_softedge'
shapename = 'noise' #'Shape2'
datafile = os.path.join(datadir, shapename+'_Coordinates_Cart_'+smoothed+'.csv')
connfile = os.path.join(datadir, shapename+'_Connectivity.csv')
maskfile = os.path.join(datadir, shapename+'_Mask'+dilated+'.csv')

# Material properties
mu0 = 300/3; nu0 = 0.499;
r0 = 5;           # microns
Tf_err = 0.1;     # unit in mu (100Pa)
# Spherical Harmonics Analysis Settings
lJmax = 20; lKmax = lJmax; lmax_plot = 60;

# initial guess settings
AKinitfile = 'final_AK.npy' #'AK_init.npy' #'AK_lmax20_init.npy'
ldamp_lo = lJmax//2; ldamp_hi = 7*lJmax//8;
lwork = 15
init_guess_type = 'ur-only' #'zero' #'ur-only' # or 'true' or 'Tr-only'
# damp the high order coefficients
lv, _ = LM_list(lJmax); lv_ones = np.ones_like(lv);
lv_lim = np.minimum(np.maximum(lv, ldamp_lo), ldamp_hi)
ldamp = np.sin(np.pi/2*(ldamp_hi-lv_lim)/(ldamp_hi-ldamp_lo))**2
AKdamp = np.tile(ldamp, 3)

# regularizations
myalpha = 1     # traction magnitude
mybeta  = 1e-4  # coefficient magnitude

# program switches
plot_figure = False
node_face_dist_on = None # not used
opt_proc = [2, ]; myord = 1; # 2: sol2dr; myord: p-norm order (1-mean value)

# minimization settings
N_period = 5
maxiter_per_period = 100
CG_gtol = 1

from scipy.optimize import OptimizeResult #, anneal
def noop_min(fun, x0, args, **options):
    return OptimizeResult(x=x0, fun=fun(x0), success=True, nfev=1)
minimizer = noop_min #'Powell' #noop_min #'CG'
minimizer_config = {} #{'maxiter': maxiter_per_period, 'disp': True, 'gtol': CG_gtol}
#{'xtol': 10, 'ftol': 0.1, 'disp':True} #{'disp': True} #

# dump files for minimization
savename = shapename #+('_a%.0e_b%.0e'%(myalpha, mybeta))+('_lmax%d'%lJmax)+smoothed+dilated
AKfile = 'AK_iter_'+savename+'.npy'
fvfile = 'fv_'+savename+'.txt'

# settings for loading \hat{U}, \hat{T} coefficients
shtype = 'reg'
coeff_dir = os.path.join(module_path, 'lmax60modes')

#################################################################################
mu = 1.; nu = nu0;
Umodes = loadmat(os.path.join(coeff_dir,'Umodes.mat'))
Umodes = (Umodes['U1'+shtype], Umodes['U0'+shtype])
fullDmat = calUmode(Umodes, mu, nu)
Dmat = generate_submat(mu, nu, fullDmat, lKmax, lJmax, kJ=3).tocsc()

Tmodes = loadmat(os.path.join(coeff_dir,'Tmodes.mat'))
Tmodes = (Tmodes['T1'+shtype], Tmodes['T2'+shtype], Tmodes['T3'+shtype], Tmodes['T0'+shtype])
fullCmat = calSmode(Tmodes, mu, nu)
Cmat = generate_submat(mu, nu, fullCmat, lKmax, lJmax, kJ=3).tocsc()

## Define arbitrary force field

In [ ]:
# Shape of the initial guess U1avec0: Xt = X0 + u1a
latsdeg, lonsdeg = pyshtools.expand.GLQGridCoord(lJmax)
lon = np.deg2rad(lonsdeg)
colat = np.deg2rad(90-latsdeg)
PHI, THETA = np.meshgrid(lon, colat)
R = np.ones_like(PHI)*r0
X,Y,Z = SphCoord_to_CartCoord(R, THETA, PHI)
X0 = np.stack([X,Y,Z], axis=-1)
N = X0/r0

pLeft = np.array([0, -r0, 0]); pRight = np.array([0, r0, 0]);
rcut = r0/3; mag = 4; dilation = 2;
distLeft = rcut - np.linalg.norm(X0 - pLeft.reshape(1, 1, 3), axis=-1)
distRight= rcut - np.linalg.norm(X0 - pRight.reshape(1, 1, 3),axis=-1)
regionLeft = 1/(1 + np.exp(-distLeft*mag))
regionRight = 1/(1 + np.exp(-distRight*mag))

pF = 0.1
T_usr_mesh_init = np.zeros_like(X0, dtype=np.complex)
F_mesh = np.zeros_like(X0, dtype=np.complex)
F_mesh[..., 1] = (pF+0j)*regionLeft - (pF+0j)*regionRight
T_usr_mesh_init = F_mesh

In [ ]:
T_usr_vec = SHmesh2Vec(T_usr_mesh_init)# *AKdamp
if plot_figure:
    fig, axs = visSHVec(T_usr_vec*mu0, lmax_plot=lmax_plot, SphCoord=True, Complex=True,
                       config_quiver=(2, 3, 'k', 500), lonshift=180, figsize=(6,3))#, n_vrange=(-10,0))

    fig, ax = plt.subplots(1, 1)
    ax.plot(np.abs(T_usr_vec))
    plt.show()

T_usr_mesh = SHVec2mesh(T_usr_vec, lmax=lJmax, SphCoord=False, Complex=True)
T_usr_mesh_sph = SHVec2mesh(T_usr_vec, lmax=lJmax, SphCoord=True, Complex=True)

In [ ]:
Uvec = Tvec2Uvec(T_usr_vec, Cmat, Dmat)

umesh1_fine = SHVec2mesh(Uvec*r0, lmax=lmax_plot, SphCoord=False, Complex=True)
tmesh1_fine = SHVec2mesh(T_usr_vec*mu0, lmax=lmax_plot, SphCoord=False, Complex=True)
if plot_figure:
    fig, ax = visSH3d(umesh1_fine, cmesh=tmesh1_fine, r0=r0, show=False, 
                      elevation=15, azimuth=45, surface=True, figsize=(4, 4))
    ax.set_xlim(-r0, r0)
    ax.set_ylim(-r0, r0)
    ax.set_zlim(-r0, r0)
    plt.show()

In [ ]:
if plot_figure:
    fig, ax = visSHVec(Uvec*r0, lmax_plot=lmax_plot, SphCoord=True, Complex=True,
                       config_quiver=(2, 3, 'k', 10), lonshift=180, figsize=(6,3))
    fig, ax = plt.subplots(1, 1)
    ax.plot(np.abs(Uvec))
    ax.set_title('$U^{(K)}$')
    plt.show()

In [ ]:
print(np.abs(T_usr_vec - Uvec2Tvec(Uvec, Cmat, Dmat)).max())

latsdeg, lonsdeg = pyshtools.expand.GLQGridCoord(lmax_plot)
lon = np.deg2rad(lonsdeg)
colat = np.deg2rad(90-latsdeg)
LONS, LATS = np.meshgrid(lonsdeg, latsdeg)
PHI, THETA = np.meshgrid(lon, colat)
R = np.ones_like(PHI)*r0
X,Y,Z = SphCoord_to_CartCoord(R, THETA, PHI)
VX0 = np.stack([X,Y,Z], axis=-1)
print(VX0.shape)

Vp = (VX0 + umesh1_fine).reshape(-1, 3)
TcLeft = (rcut+dilation) > np.linalg.norm(VX0 - pLeft.reshape(1, 1, 3), axis=-1)
TcRight= (rcut+dilation) > np.linalg.norm(VX0 - pRight.reshape(1, 1, 3), axis=-1)
TfRegion = np.logical_not(np.logical_or(TcLeft, TcRight))

In [ ]:
if plot_figure:
    %matplotlib notebook
    fig, axs = visSHVec(T_usr_vec*mu0, lmax_plot=lmax_plot, SphCoord=True, Complex=True,
                       config_quiver=(2, 3, 'k', 1000), lonshift=180, figsize=(6,3), n_vrange=(-10,0))

    axs[0].contour(LONS, LATS, TfRegion, [0.5,], colors='k', linewidths=1)
    axs[1].contour(LONS, LATS, TfRegion, [0.5,], colors='k', linewidths=1)
    plt.show()

Tfv = TfRegion.flatten()

# save datafile and maskfile
np.savetxt(datafile, Vp, delimiter=',')
np.savetxt(maskfile, Tfv)
AK_true = lsqr(Cmat, T_usr_vec.T, atol=0, btol=0, conlim=0)[0]
np.save('AK_true', AK_true)

In [ ]:
#### Plot the geometry (Vp) ####
if plot_figure:
    fig = plt.figure()#figsize=(16,16))
    ax = fig.add_subplot(111, projection='3d')

    # nTfv = np.logical_not(Tfv)
    Tf_id = Tfv < 0.1
    nTf_id= np.logical_not(Tf_id)
    ax.scatter3D(Vp[Tf_id, 0], Vp[Tf_id, 1], Vp[Tf_id, 2])
    ax.scatter3D(Vp[nTf_id, 0], Vp[nTf_id, 1], Vp[nTf_id, 2])

    ax.view_init(azim=90, elev=30)
    #ax.set_aspect(1)
    plt.show()

Then we determine the original radius of the particle, assuming the particle is incompressible. The total volume can be estimated by adding the volume of the tetrahedrons. The volume of a tetrahedron is calculated as:

$$
V_{0123}=\frac{1}{6}
\begin{vmatrix}
 x_1 & y_1 & z_1 & 1\\ 
 x_2 & y_2 & z_2 & 1\\ 
 x_3 & y_3 & z_3 & 1\\ 
 0 & 0 & 0 & 1\\ 
\end{vmatrix}
$$

We need to solve the reverse problem of a deformed shape. We will try the following methods to tackle this problem:

1. Assume $r$-direction deformation only, using the solution as initial guess to the optimization
2. LSQ solving coeffs of the SH solutions for fitting both the traction-free boundary and the shape

Conversion between complex and real spherical harmonics, for $m>0$

$$
f_{lm} = \left[f_l^m+(-1)^mf_l^{-m}\right]/\sqrt{2}\\
f_{l-m}=i\left[f_l^m-(-1)^mf_l^{-m}\right]/\sqrt{2}\\
f_l^m = (f_{lm}-if_{l-m})/\sqrt{2}\\
f_l^{-m} = (-1)^m(f_{lm}+if_{l-m})/\sqrt{2}
$$

for $m=0$:

$$
f_{l0} = f_l^0
$$


In [ ]:
from Case05_utilities import AKcomp2real, AKreal2comp
print(np.abs(AKreal2comp(AKcomp2real(AK_true, lJmax), lJmax) - AK_true).max())

## 1. Obtain the initial guess

First thing to try is assuming that the deformation only happens on $r$-direction.

In [ ]:
%matplotlib inline
# Vp -> ur (radial displacement) -> u1 (x,y,z)
Vr, Vthe, Vphi = CartCoord_to_SphCoord(Vp[...,0], Vp[...,1], Vp[...,2])
Vphi[Vphi < 0] += 2*np.pi
Vlat = 90-np.rad2deg(Vthe)
Vlon = np.rad2deg(Vphi)

ur = (Vr - r0)/r0

# plot 2d map of the radial displacement
if plot_figure:
    pVlon = Vlon - 180
    pVlon[pVlon < 0] += 360
    fig,ax = plt.subplots(1,1,figsize=(6,2.5))
    im = ax.tripcolor(pVlon, Vlat, ur)
    fig.colorbar(im)
    ax.axis('equal')
    ax.set_xlim(0, 360)
    ax.set_ylim(-90, 90)
    plt.show()

In [ ]:
if not os.path.exists(AKinitfile):
    if init_guess_type == 'true':
        AK_init = lsqr(Cmat, T_usr_vec.T, atol=0, btol=0, conlim=0)[0]
        U1avec0 = Dmat.dot(AK_init)
    elif init_guess_type == 'ur-only':
        print('Calculate AKinit from assumption of ur only...')
        # guess from radial displacement only
        urcilm, chi2 = pyshtools.expand.SHExpandLSQ(ur, Vlat, Vlon, lmax=lJmax)
        print(chi2)
        urcoeffs = pyshtools.SHCoeffs.from_array(urcilm)
        urcoeffs.info()
        spec_before = urcoeffs.spectrum(unit='per_lm')
        if plot_figure:
            fig, ax = urcoeffs.plot_spectrum(unit='per_lm')
        # visualize ur map
        urgrid = urcoeffs.pad(lmax=lmax_plot).expand('GLQ')
        if plot_figure:
            urgrid.info()
            urgrid_plot = urgrid.copy()
            urgrid_plot.data = np.roll(urgrid.data, lmax_plot+1, axis=1)
            fig, ax = urgrid_plot.plot()
        # Convert Back to ux,uy,uz on regular mesh
        Q = TransMat(lJmax=urgrid.lmax)
        print('Size of transfer matrix:', Q.shape)
        u1a = urgrid.data[:,:,np.newaxis]*Q[:, :, 0, :]
        print('Size of Cartesian representation of displacement:', u1a.shape)
        # decompose the displacement field into spherical harmonic coefficients
        U1avec0 = [None for _ in range(3)]
        for k in range(3):
            u1agrid = pyshtools.SHGrid.from_array(u1a[...,k].astype(np.complex), grid='GLQ')
            u1acoeffs = u1agrid.expand().pad(lmax=lJmax)
            U1avec0[k] = SHCilmToVector(u1acoeffs.to_array())
        U1avec0 = np.hstack(U1avec0)
        # damp the high order coefficients
        lv, _ = LM_list(lJmax); lv_ones = np.ones_like(lv);
        lv_lim = np.minimum(np.maximum(lv, ldamp_lo), ldamp_hi)
        ldamp = np.sin(np.pi/2*(ldamp_hi-lv_lim)/(ldamp_hi-ldamp_lo))**2
        AKdamp = np.tile(ldamp, 3)
        AK_init = spsolve(Dmat, U1avec0.T)*AKdamp
        if plot_figure:
            plt.semilogy(np.abs(spsolve(Dmat, U1avec0.T)));
            plt.semilogy(np.abs(AK_init)); plt.show();
    elif init_guess_type == 'Tr-only':
        T_mesh_norm = np.sum(F_mesh*N, axis=-1)[...,np.newaxis]*N
        T_vec_norm = SHmesh2Vec(T_mesh_norm)
        AK_init = lsqr(Cmat, T_vec_norm.T, atol=0, btol=0, conlim=0)[0]
        U1avec0 = Dmat.dot(AK_init)
    elif init_guess_type == 'zero':
        AK_init = np.zeros((3, (lJmax+1)**2), dtype=np.complex).flatten()
        U1avec0 = np.zeros_like(AK_init, dtype=np.complex)
else: # load the initial guess from a file
    print('load AKinit from file:', AKinitfile)
    AK_tmp = np.load(AKinitfile).reshape(3, -1)
    AK_init = np.zeros((3, (lJmax+1)**2), dtype=np.complex)
    lJmax_init = np.int(np.sqrt(AK_tmp.shape[1]) - 1)
    # padding
    if lJmax_init > lJmax:
        AK_init = AK_tmp[:, :((lJmax+1)**2)]
    else:
        AK_init[:, :((lJmax_init+1)**2)] = AK_tmp
    AK_init = AK_init.flatten()
    lv, _ = LM_list(lJmax); lv_ones = np.ones_like(lv);
    lv_lim = np.minimum(np.maximum(lv, ldamp_lo), ldamp_hi)
    ldamp = np.sin(np.pi/2*(ldamp_hi-lv_lim)/(ldamp_hi-ldamp_lo))**2
    U1damp = np.tile(ldamp, 3)**4
    U1avec0 = Dmat.dot(AK_init)*U1damp
    AK_init = spsolve(Dmat, U1avec0.T)

if plot_figure:
    print('Coefficients of the Initial Guess in Solution Space...AK')
    plt.plot(np.abs(AK_init))
    plt.ylim(0,0.1)
    plt.show()
    print('Displacement in Spherical Coordinates...')
    fig, ax = visSHVec(U1avec0*r0, lmax_plot=lmax_plot, SphCoord=True, Complex=True, #s_vrange=(0,0.01),
                       config_quiver=(2, 3, 'k', 10), lonshift=180, figsize=(6,3))
    print('SH Coefficients of the Initial Guess Displacement...U1avec0')
    fig, ax = plt.subplots(1, 1)
    ax.plot(np.abs(U1avec0))
    ax.set_ylim(0,0.1)
    T1avec0 = Uvec2Tvec(U1avec0, Cmat, Dmat)
    print('Traction in Spherical Coordinates...')
    fig, ax = visSHVec(T1avec0*mu0, lmax_plot=lmax_plot, SphCoord=True, Complex=True,
                       n_vrange=(-100, 100), s_vrange=(0, 50),
                       config_quiver=(2, 3, 'k', 500), lonshift=180, figsize=(6,3))
    print('SH Coefficients of the Initial Guess Displacement...T1avec0')
    fig, ax = plt.subplots(1, 1)
    ax.plot(np.abs(T1avec0))
    ax.set_ylim(0,0.1)
    plt.show()

## 2. LSQ solving SH coeffs for displacement field

Obviously, the decomposition is not satisfactory. It is not reasonable to assume the deformation is only on $r$-direction. In this section, we will try to optimize SH coeffs, so that the deformed shape is closest to the data. Notice that the integral of a spherical harmonic function on the sphere surface is:

$$
\int_0^{2\pi}\!\int_0^{\pi}Y_l^m(\theta,\varphi)\sin\theta d\theta d\varphi = 4\pi\delta_{l0}\delta_{m0}
$$

Therefore, only the $Y_0^0$ term controls the rigid body translation (constant). If we only impose higher mode spherical harmonics, there will be no rigid body motion.

### 2.1 Developing shape-difference function and neighbor list

$$ \Delta u = \langle u_r(\theta, \phi) - u_t(\theta, \phi)\rangle_{(\theta,\phi)} $$

In [ ]:
# Shape of the initial guess U1avec0: Xt = X0 + u1a
latsdeg, lonsdeg = pyshtools.expand.GLQGridCoord(lJmax)
lon = np.deg2rad(lonsdeg)
colat = np.deg2rad(90-latsdeg)
PHI, THETA = np.meshgrid(lon, colat)
R = np.ones_like(PHI)
X,Y,Z = SphCoord_to_CartCoord(R, THETA, PHI)
X0 = np.stack([X,Y,Z], axis=-1)

Xt = X0 + SHVec2mesh(U1avec0, lmax=lJmax, SphCoord=False, Complex=True)

### Develop the interpolation function for $u_r(\theta,\varphi)$ from data

In [ ]:
if (2 in opt_proc) or (3 in opt_proc):
    urcilm_interp, chi2 = pyshtools.expand.SHExpandLSQ(ur, Vlat, Vlon, lmax=lJmax+10)
    print(chi2)

    ucoeff_interp = pyshtools.SHCoeffs.from_array(urcilm_interp)
    urgrid_interp = ucoeff_interp.expand('GLQ')

    if plot_figure:
        fig, ax = ucoeff_interp.plot_spectrum(unit='per_lm', show=False)
        urgrid_interp.info()

    from scipy.interpolate import RectBivariateSpline

    lats = urgrid_interp.lats(); lons = urgrid_interp.lons()
    lats_circular = np.hstack(([90.], lats, [-90.]))
    lons_circular = np.append(lons, 360)
    LONS, LATS = np.meshgrid(lons_circular, lats_circular)
    xmesh = urgrid_interp.to_array().copy()
    fpoints = np.zeros_like(LONS)
    fpoints[1:-1, :-1] = xmesh
    fpoints[0, :] = np.mean(xmesh[0,:], axis=0)  # not exact !
    fpoints[-1, :] = np.mean(xmesh[-1,:], axis=0)  # not exact !
    fpoints[1:-1, -1] = xmesh[:, 0]
    print(LATS.shape, LONS.shape, fpoints.shape)
    print(lats_circular.shape, lons_circular.shape)
    f_interp = RectBivariateSpline(lats_circular[::-1], lons_circular, fpoints[::-1, ], kx=1, ky=1)

    ## Test the interpolation algorithm

    tic = time.time()
    ur_interp = f_interp.ev(Vlat, Vlon)
    toc = time.time()
    print(f_interp.get_coeffs().shape)
    print(ur_interp.shape, toc-tic)
    print(ur_interp - ur)

    if plot_figure:
        fig, axs = plt.subplots(1,1,figsize=(6,2.5))
        im = axs.tripcolor(pVlon, Vlat, ur)
        fig.colorbar(im)
        axs.axis('equal')
        axs.set_xlim(0, 360)
        axs.set_ylim(-90, 90)
        fig, axs = plt.subplots(1,1,figsize=(6,2.5))
        im = axs.tripcolor(pVlon, Vlat, (ur_interp-ur)*r0)
        fig.colorbar(im)
        axs.axis('equal')
        axs.set_xlim(0, 360)
        axs.set_ylim(-90, 90)
        #plt.show()

### 2.2  From `uvec` to the shape difference

After we obtain the neighbor list, the shape can be obtained from the real SH vector `uvec`. This can be used for the optimization of the shape.

In [ ]:
from Case05_utilities import coeffs2dr
# detailed implementation in Case05_utilities.py

lat_weights = np.sin(THETA)
vert_weight = np.array([1,1,1./3])
l_list, m_list = LM_list(lJmax)

### 2.3 Target function including shape and traction

In [ ]:
# Define weights and traction free region
x0 = Xt;
dist2mat = np.linalg.norm(x0[..., np.newaxis, :] - Vp/r0, axis=-1)
arg_list_x = dist2mat.argmin(axis=-1)
if dilated == '_softedge':
    isTfv = mask[arg_list_x]
else:
    isTfv = Tfv[arg_list_x]

Q = TransMat(lJmax=lJmax)           # transformation matrix from Cartesian to Spherical Coordinates
l_list, m_list = LM_list(lJmax)
l_weight = l_list - lwork; l_weight[l_weight < 0] = 0; l_weight = (np.exp(l_weight) - 1);
A_weight = np.tile(l_weight, 3)

print("A_weight.shape =",A_weight.shape)
if plot_figure:
    plt.plot(A_weight); 
    #plt.show()

In [ ]:
from Case05_utilities import sol2dr, sol2dist_verbose, sol2dist_update

We use the solution we obtained from Section 1 `Uvec_real` for the testing:

In [ ]:
print('Calculate node-node distances without and with weights...')

tic = time.time()
dist_test = coeffs2dr(U1avec0, f_interp=f_interp, lmax=lJmax, X0=X0, Complex=True, debug=True)
toc = time.time()
print("minimum distances squared from all mesh points:")
norm2 = np.sum(dist_test**2, axis=-1)
print(norm2[:,0])
print("mean of minimum distances squared:", norm2.mean())
if plot_figure:
    fig, ax = plotfv(norm2, show=False, lonshift=180)
    ax.set_title('minimum distance squared on mesh points')
    plt.show()
print("mean of minimum distances",myord*2,"-norm:", np.linalg.norm(norm2.flatten(), ord=myord)/(lJmax+1)/(2*lJmax+1))

tic = time.time()
dist_test = coeffs2dr(U1avec0, f_interp=f_interp, lmax=lJmax, X0=X0, Complex=True, norm_order=myord)
toc = time.time()
print('compare only neighbor list: dist = %.8f, time = %.4f'%(dist_test, toc-tic))

print('Calculate target function using alpha=',0,'beta =',mybeta)
tic = time.time()
dist_test = sol2dr(AK_init, Cmat, Dmat, alpha = 0, beta = mybeta, isTfv=isTfv, f_interp=f_interp, separate=False, lmax=lJmax, X0=X0, l_weight=A_weight, norm_order=myord)
toc = time.time()
print('compare only neighbor list: dist = %.8f, time = %.4f'%(dist_test, toc-tic))

print('Calculate target function using alpha=',myalpha,'beta =',mybeta)
tic = time.time()
dist_test = sol2dr(AK_init, Cmat, Dmat, alpha = myalpha, beta = mybeta, isTfv=isTfv, f_interp=f_interp, separate=False, lmax=lJmax, X0=X0, l_weight=A_weight, norm_order=myord)
toc = time.time()
print('compare only neighbor list: dist = %.8f, time = %.4f'%(dist_test, toc-tic))
dist_test = sol2dr(AK_init, Cmat, Dmat, alpha = myalpha, beta = mybeta, isTfv=isTfv, f_interp=f_interp, separate=True, lmax=lJmax, X0=X0, l_weight=A_weight, norm_order=myord)
verb = sol2dist_verbose(dist_test, mu0=mu0, r0=r0)

### 2.4 Solution by minimizing the target function

Then we can optimizing the solution by minimizing the distance:

In [ ]:
from Case05_utilities import minimize_AK

### 2.5 The optimization process, use node-node and node-face interchangably

In [ ]:
#from scipy.optimize import basinhopping

# minimizer = basinhopping
# minimizer_config = {'method': 'Powell', 'options':{'maxiter': maxiter_per_period, 'disp': True}}

if (2 in opt_proc):
    args_dr = (Cmat, Dmat, myalpha, mybeta, isTfv, f_interp, lJmax, X0, 
               lat_weights, vert_weight, A_weight, myord)
    iter_config_sol2dr = {'N_period': N_period, 'minimizer': minimizer, 'minimizer_config': minimizer_config,
                          'n_update': 0}

AK_iter = AK_init #+ np.mean(AK_init)*np.random.rand(*AK_init.shape)*10
for i_proc in opt_proc:
    if i_proc > 1:
        target = sol2dr; args = args_dr; iter_config = iter_config_sol2dr;
    AK_iter, funval = minimize_AK(AK_iter, target, args, iter_config, verbose_args=(r0, mu0), use_basinhopping=100,
                                  verbose=sol2dist_verbose, AKfile=AKfile, fvfile=fvfile)

In [ ]:
import glob
filelist = glob.glob('AK_final_??.npz')
nfile = len(filelist)
np.savez('AK_final_%02d.npz'%nfile, AK_iter=AK_iter, alpha=myalpha, beta=mybeta, N=N_period, fv_iter=funval)

In [ ]:
# To do:
# 1. Save solution: AK_iter (overwrite), funval (append) to file every step (done)
# 2. Every run has its own folder (done)
# 3. add option for node-face distance in optimization (done)
# 4. wrap the minimization into function (done)
# 5. spherical harmonics shape-difference representation
#    5.1. interpolation from irregular ur data (done)
#    5.2. new coeffs2dist (done)
# 6. smoothed edge of traction free boundary
#    6.1. smooth the edge (Case05-Hydrogel_Cell_Interaction-smoothed_mask.ipynb) (done)
#    6.2. optimize with smoothed edge (done)
# 7. 

# Uncomment the following line if want to save AKfile from memory
# np.save(AKfile, AK_min.x)
# plt.plot(AK_min.x); plt.show()
# Uncomment the following line if want to save fvfile from memory
# np.savetxt(fvfile, np.hstack([np.arange(plot_funval.shape[0]).reshape(-1,1), plot_funval]), fmt='%8d %e %e %e')